In [23]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("declare-lab/flan-alpaca-base")
model = AutoModelForSeq2SeqLM.from_pretrained("declare-lab/flan-alpaca-base")

def alpaca_embeddings(text):
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt")

    # Generate dummy decoder input ids
    dummy_decoder_input_ids = torch.zeros((1, 1), dtype=torch.long)

    # Get the embeddings from the model
    with torch.no_grad():
        outputs = model(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            decoder_input_ids=dummy_decoder_input_ids,
        )

    embeddings = outputs.encoder_last_hidden_state
    return embeddings

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl")

def t5_embeddings(text):
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt")

    # Generate dummy decoder input ids
    dummy_decoder_input_ids = torch.zeros((1, 1), dtype=torch.long)

    # Get the embeddings from the model
    with torch.no_grad():
        outputs = model(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            decoder_input_ids=dummy_decoder_input_ids,
        )

    embeddings = outputs.encoder_last_hidden_state
    return embeddings

In [36]:
len(t5_embeddings("Ich bin ein Berliner")[0][0])

1024

In [ ]:
def get_sentence_embeddings(sentence):
    prompt = "Represent the theme and the authors opinion in the document for clustering: '{}'".format(sentence)
    input_ids = torch.tensor(tokenizer.encode(prompt, return_tensors='pt'))
    if torch.cuda.is_available():
        input_ids = input_ids.to('cuda')
        model.to('cuda')
    with torch.no_grad():
        last_hidden_states = model.generate(input_ids)[0]
    last_hidden_states = last_hidden_states.cpu().numpy()
    return last_hidden_states